# Use of Custom Callbacks

## Data Preparation
1. The dataset was successfully downloaded from [this link](https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing).
2. The data was formatted and preprocessed as per the requirements. The model was trained to classify data into the categories shown in the image below:

![Classification Categories](https://i.imgur.com/33ptOFy.png)

---

## Implementation Details

### Custom Callback Function
- A custom callback function was implemented to print the **Micro F1 Score** and **AUC Score** after each epoch. This allowed real-time monitoring of performance metrics during training.

---

### Model Saving
- The model was saved at the end of each epoch where validation accuracy improved over the previous epoch. This ensured we retained the best-performing model.

---

### Learning Rate Decay
- The learning rate was decayed dynamically based on the following conditions:
  - **Condition 1:** If validation accuracy decreased compared to the previous epoch, the learning rate was reduced by 10%.
  - **Condition 2:** For every 3rd epoch, the learning rate was decayed by 5%.
  
This strategy helped stabilize training and improved convergence.

---

### Training Termination
- The training was terminated automatically if:
  1. Any NaN values (weights or loss) were encountered during training.
  2. Validation accuracy did not improve over the last 2 consecutive epochs.

---

### TensorBoard Analysis
- TensorBoard was utilized to visualize the training process, analyze gradients, and evaluate the model's performance. Screenshots of the TensorBoard outputs were saved for documentation.

---

### Loss Function
- **Categorical Cross-Entropy** was used as the loss function for all models.

---

## Models and Observations

### Model-1
- **Architecture:**
  - Activation Function: `tanh` (for all layers except the output layer)
  - Optimizer: `SGD` with momentum
  - Weight Initializer: `RandomUniform(0,1)`
  
- **Observations:**
  - The use of `tanh` activation helped smooth the gradients, but training was slightly slower compared to ReLU.
  - Achieved stable validation accuracy after 15 epochs.

---

### Model-2
- **Architecture:**
  - Activation Function: `relu` (for all layers except the output layer)
  - Optimizer: `SGD` with momentum
  - Weight Initializer: `RandomUniform(0,1)`
  
- **Observations:**
  - The `relu` activation led to faster convergence compared to `tanh`.
  - Validation accuracy slightly fluctuated but showed consistent improvement overall.

---

### Model-3
- **Architecture:**
  - Activation Function: `relu` (for all layers except the output layer)
  - Optimizer: `SGD` with momentum
  - Weight Initializer: `he_uniform`
  
- **Observations:**
  - The `he_uniform` initializer worked well with `relu` activation, resulting in faster training and improved generalization.
  - Validation accuracy peaked earlier compared to the other models.

---

### Model-4
- **Experimentation:**
  - Various hyperparameters and configurations were tested to optimize performance.
  - Achieved the best F1 Score and accuracy by adjusting the learning rate decay and batch size.
  
- **Observations:**
  - The flexibility in tuning parameters resulted in significant improvements in both F1 and AUC scores.
  - This model performed the best among the four.

---

## Conclusion
- All models were successfully trained, and their performance metrics were analyzed.
- The optimal model was identified based on validation accuracy, F1 Score, and AUC Score.
- TensorBoard visualizations provided valuable insights into gradient flow and learning dynamics.


In [ ]:
#importing libraries
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
tf.__version__

'2.2.0'

#1

In [ ]:
d=pd.read_csv('/content/drive/My Drive/Colab Notebooks/question 20/data.csv') #reading the csv file

In [ ]:
d.head(5)

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [ ]:
d.shape

(20000, 3)

In [ ]:
features = ['f1', 'f2'] #dividing features and label

In [ ]:
l=len(d)
tr_l=int(0.8*l)
te_l=int(0.2*l)

In [ ]:
train_data_pd=d.iloc[:(tr_l)]

In [ ]:
train_data_pd.shape

(16000, 3)

In [ ]:
y_train=train_data_pd['label'].to_numpy() #dividing into x_train and y_train

In [ ]:
X_train=train_data_pd[features].to_numpy()

In [ ]:
test_data_pd=d.iloc[:(te_l)]

In [ ]:
test_data_pd.shape

(4000, 3)

In [ ]:
y_test=test_data_pd['label'].to_numpy()##dividing into x_train and y_train

In [ ]:
X_test=test_data_pd[features].to_numpy()

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
#importing some libraries
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

#2 and 3

In [ ]:
#using sklearn to ccompute the auc and  f1 score b/w y_perd and y_true
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import f1_score, roc_curve,auc
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs=None): #when an epoch ends
        val_predict=self.model.predict(self.x_val)
        #print(val_predict)
        v=[]
        for j in val_predict:
            if j<0.5:
                j=0
            else: j=1
            v.append(j)
        #print(v)
        val_targ = self.y_val
        #print(val_targ)
        _val_f1 = f1_score(val_targ, v) #calculating f1 score
        fpr,tpr,t=roc_curve(val_targ,v,pos_label=1)
        auc_score=auc(fpr,tpr) #calculating auc_score

        print(" F1 Score : {} ,  AUC Score : {} ".format(_val_f1,auc_score))


metrics = Metrics(training_data=(X_train, y_train),validation_data=(X_test, y_test))

Using TensorFlow backend.


In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=1,callbacks=metrics) #fit models with above callback

Epoch 1/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6072 - accuracy: 0.6725 - val_loss: 0.6076 - val_accuracy: 0.6737
Epoch 2/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6063 - accuracy: 0.6691 - val_loss: 0.6118 - val_accuracy: 0.6675
Epoch 3/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6066 - accuracy: 0.6703 - val_loss: 0.6081 - val_accuracy: 0.6743
Epoch 4/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6054 - accuracy: 0.6713 - val_loss: 0.6142 - val_accuracy: 0.6675
Epoch 5/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6065 - accuracy: 0.6713 - val_loss: 0.6082 - val_accuracy: 0.6718
Epoch 6/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6057 - accuracy: 0.6729 - val_loss: 0.6087 - val_accuracy: 0.6708
Epoch 7/15
16000/16000 [==============================] - 22s 1ms/step - loss: 0.6061 - accuracy: 0.6694 - val_loss: 0

#4


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint #importing bultin callback

In [ ]:
#defining the file path for saving the weights
filepath="/content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  save_weights_only=False, verbose=1, save_best_only=True, mode='max',save_freq='epoch')

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=1,callbacks=checkpoint)

Epoch 1/15
15999/16000 [============================>.] - ETA: 0s - loss: 0.6930 - accuracy: 0.5053
Epoch 00001: val_accuracy improved from -inf to 0.50225, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-01-0.5023.hdf5
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6930 - accuracy: 0.5053 - val_loss: 0.6915 - val_accuracy: 0.5023
Epoch 2/15
15965/16000 [============================>.] - ETA: 0s - loss: 0.6899 - accuracy: 0.5179
Epoch 00002: val_accuracy improved from 0.50225 to 0.54400, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-02-0.5440.hdf5
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6899 - accuracy: 0.5181 - val_loss: 0.6875 - val_accuracy: 0.5440
Epoch 3/15
15966/16000 [============================>.] - ETA: 0s - loss: 0.6859 - accuracy: 0.5311
Epoch 00003: val_accuracy did not improve from 0.54400
16000/16000 [==============================] - 21s 1

#5


In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import keras.backend as K

In [ ]:
# import math
# def step_decay(epoch):
#    initial_lrate = 0.001
#    drop = 0.95
#    epochs_drop = 3
#    lrate = initial_lrate * math.pow(drop,
#            math.floor((1+epoch)/epochs_drop))
#    return lrate
# lrate = LearningRateScheduler(step_decay)

In [ ]:
#custom callback for decreasing the learning rate for given conditions
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.accuracy = []
       self.l_r=[]
    def on_epoch_end(self, epoch, logs={}):

        self.accuracy.append(logs.get('val_accuracy'))

        if epoch>0:
            #print( '   epoch ', epoch)


            if self.accuracy[epoch]<self.accuracy[epoch-1] :
                    #print('e',self.accuracy[epoch])
                    #print('e-1',self.accuracy[epoch-1])
                (model.optimizer.lr)=model.optimizer.lr*(0.90)
            if (epoch+1)%3==0:
                model.optimizer.lr=model.optimizer.lr*(0.95)
        self.l_r.append(K.eval(model.optimizer.lr))
        print(' learning rate : ', self.l_r[epoch])


In [ ]:
loss_history = LossHistory()

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=1,callbacks=loss_history)

Epoch 1/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6953 - accuracy: 0.5034 - val_loss: 0.6932 - val_accuracy: 0.4978
Epoch 2/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6936 - accuracy: 0.4972 - val_loss: 0.6939 - val_accuracy: 0.5023
Epoch 3/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6935 - accuracy: 0.5036 - val_loss: 0.6929 - val_accuracy: 0.5428
Epoch 4/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6931 - accuracy: 0.5025 - val_loss: 0.6920 - val_accuracy: 0.5023
Epoch 5/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6909 - accuracy: 0.5241 - val_loss: 0.6880 - val_accuracy: 0.5428
Epoch 6/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6819 - accuracy: 0.5768 - val_loss: 0.6714 - val_accuracy: 0.6155
Epoch 7/15
16000/16000 [==============================] - 20s 1ms/step - loss: 0.6571 - accuracy: 0.6239 - val_loss: 0

#6

In [ ]:
# class TerminateNaN(tf.keras.callbacks.Callback):

#     def on_epoch_end(self, epoch, logs={}):
#         loss = logs.get('loss')
#         if loss is not None:
#             if np.isnan(loss) or np.isinf(loss):
#                 print("Invalid loss and terminated at epoch {}".format(epoch))
#                 self.model.stop_training = True

In [ ]:
#when any NaN values occur i.e in weights or call the built in callback will work
t=tf.keras.callbacks.TerminateOnNaN()

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=50,callbacks=t)

Epoch 1/20
320/320 [==============================] - 1s 2ms/step - loss: 0.7210 - accuracy: 0.5035 - val_loss: 0.6951 - val_accuracy: 0.5023
Epoch 2/20
320/320 [==============================] - 1s 2ms/step - loss: 0.6935 - accuracy: 0.5035 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 3/20
320/320 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.4978
Epoch 4/20
320/320 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.4997 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/20
320/320 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 6/20
320/320 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.5012 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 7/20
320/320 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5018 - val_loss: 0.6932 - val_accuracy: 0.5023
Epoch 

#7

In [ ]:
#importing the ealy stopping for val_accuracy
ES=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=2, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=1,callbacks=ES)

Epoch 1/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6952 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 2/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6937 - accuracy: 0.4968 - val_loss: 0.6933 - val_accuracy: 0.5023
Epoch 3/15
16000/16000 [==============================] - 21s 1ms/step - loss: 0.6936 - accuracy: 0.4946 - val_loss: 0.6933 - val_accuracy: 0.4978


#9

In [ ]:
#defining loss as cross entropy

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:

 model.compile(optimizer='adam',
                loss='BinaryCrossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=20)


Epoch 1/15
800/800 [==============================] - 1s 2ms/step - loss: 0.7055 - accuracy: 0.5035 - val_loss: 0.6931 - val_accuracy: 0.4863
Epoch 2/15
800/800 [==============================] - 1s 1ms/step - loss: 0.6932 - accuracy: 0.5032 - val_loss: 0.6933 - val_accuracy: 0.5023
Epoch 3/15
800/800 [==============================] - 1s 2ms/step - loss: 0.6933 - accuracy: 0.4979 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 4/15
800/800 [==============================] - 1s 1ms/step - loss: 0.6933 - accuracy: 0.4978 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/15
800/800 [==============================] - 1s 1ms/step - loss: 0.6931 - accuracy: 0.5043 - val_loss: 0.6935 - val_accuracy: 0.5023
Epoch 6/15
800/800 [==============================] - 1s 1ms/step - loss: 0.6933 - accuracy: 0.4956 - val_loss: 0.6933 - val_accuracy: 0.5023
Epoch 7/15
800/800 [==============================] - 1s 2ms/step - loss: 0.6933 - accuracy: 0.5025 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 

In [ ]:
callback_list=[metrics,checkpoint,loss_history,t,ES]

#Model 1

In [ ]:
#loading the tensorboard extension
%load_ext tensorboard
import datetime

In [ ]:
!rm -rf ./logs/  # clearing the previous logs

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # defing the log_dir

In [ ]:
#built-in tensorboard call back
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,write_grads=True)

In [ ]:
callback_list=[metrics,checkpoint,loss_history,t,ES,tensorboard_callback]

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='tanh',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(input_layer)
layer2 = Dense(4,activation='tanh',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer1)
layer3 = Dense(2,activation='tanh',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer2)
layer4 = Dense(2,activation='tanh',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer3)
layer5 = Dense(4,activation='tanh',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
 model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=1,callbacks=callback_list)


Epoch 1/20
15985/16000 [============================>.] - ETA: 0s - loss: 0.6955 - accuracy: 0.5127 F1 Score : 0.2680571649285438 ,  AUC Score : 0.5246732496333051 

Epoch 00001: val_accuracy improved from -inf to 0.52625, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-01-0.5263.hdf5
 learning rate :  0.001
16000/16000 [==============================] - 25s 2ms/step - loss: 0.6955 - accuracy: 0.5129 - val_loss: 0.6881 - val_accuracy: 0.5263
Epoch 2/20
15985/16000 [============================>.] - ETA: 0s - loss: 0.6854 - accuracy: 0.5297 F1 Score : 0.670906200317965 ,  AUC Score : 0.5361294816220029 

Epoch 00002: val_accuracy improved from 0.52625 to 0.53425, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-02-0.5343.hdf5
 learning rate :  0.001
16000/16000 [==============================] - 25s 2ms/step - loss: 0.6854 - accuracy: 0.5299 - val_loss: 0.6855 - val_accuracy: 0.5343
Epoch 3/20
15982/16000 [====

In [ ]:
%tensorboard --logdir logs/fit/ #to launch tensorboard
#screenshots have been attached in the seperate pdf file

In [ ]:
!kill 540

/bin/bash: line 0: kill: (540) - No such process


#Model 2

In [ ]:
!rm -rf ./logs/

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,write_grads=True)

In [ ]:
callback_list=[metrics,checkpoint,loss_history,t,ES,tensorboard_callback]

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='relu',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(input_layer)
layer2 = Dense(4,activation='relu',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer1)
layer3 = Dense(2,activation='relu',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer2)
layer4 = Dense(2,activation='relu',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer3)
layer5 = Dense(4,activation='relu',kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=1))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
 model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=1,callbacks=callback_list)


Epoch 1/20
15999/16000 [============================>.] - ETA: 0s - loss: 0.6738 - accuracy: 0.5738 F1 Score : 0.6778864970645793 ,  AUC Score : 0.5897606926540262 

Epoch 00001: val_accuracy improved from -inf to 0.58850, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-01-0.5885.hdf5
 learning rate :  0.001
16000/16000 [==============================] - 25s 2ms/step - loss: 0.6738 - accuracy: 0.5738 - val_loss: 0.6695 - val_accuracy: 0.5885
Epoch 2/20
15992/16000 [============================>.] - ETA: 0s - loss: 0.6513 - accuracy: 0.6184 F1 Score : 0.6813736638617239 ,  AUC Score : 0.650209791748283 

Epoch 00002: val_accuracy improved from 0.58850 to 0.64975, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-02-0.6497.hdf5
 learning rate :  0.001
16000/16000 [==============================] - 26s 2ms/step - loss: 0.6512 - accuracy: 0.6184 - val_loss: 0.6192 - val_accuracy: 0.6497
Epoch 3/20
15965/16000 [====

In [ ]:
%tensorboard --logdir logs/fit/
#to launch tensorboard
#screenshots have been attached in the seperate pdf file

#Model 3

In [ ]:
!rm -rf ./logs/

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True)

In [ ]:
callback_list=[metrics,checkpoint,loss_history,t,ES,tensorboard_callback]

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
layer2 = Dense(4,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
layer3 = Dense(2,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer2)
layer4 = Dense(2,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer3)
layer5 = Dense(4,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
 model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=1,callbacks=callback_list)


Epoch 1/20
15988/16000 [============================>.] - ETA: 0s - loss: 0.6869 - accuracy: 0.5289 F1 Score : 0.6614612381483548 ,  AUC Score : 0.546312437826866 

Epoch 00001: val_accuracy improved from 0.50225 to 0.54475, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-01-0.5447.hdf5
 learning rate :  0.001
16000/16000 [==============================] - 26s 2ms/step - loss: 0.6869 - accuracy: 0.5289 - val_loss: 0.6833 - val_accuracy: 0.5447
Epoch 2/20
15977/16000 [============================>.] - ETA: 0s - loss: 0.6837 - accuracy: 0.5336 F1 Score : 0.0 ,  AUC Score : 0.5 

Epoch 00002: val_accuracy did not improve from 0.54475
 learning rate :  0.00090000004
16000/16000 [==============================] - 26s 2ms/step - loss: 0.6837 - accuracy: 0.5332 - val_loss: 0.6828 - val_accuracy: 0.5023
Epoch 3/20
15964/16000 [============================>.] - ETA: 0s - loss: 0.6827 - accuracy: 0.5346 F1 Score : 0.6558873882442458 ,  AUC Score : 0.5491752

In [ ]:
 !kill 1257

/bin/bash: line 0: kill: (1257) - No such process


In [ ]:
%tensorboard --logdir logs/fit/
#to launch tensorboard
#screenshots have been attached in the seperate pdf file

#Model 4

In [ ]:
!rm -rf ./logs/

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
callback_list=[metrics,checkpoint,loss_history,t,ES,tensorboard_callback]

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(4,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)
layer2 = Dense(4,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer1)
layer3 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer2)
layer4 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer3)
layer5 = Dense(4,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
 optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
 model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=30,validation_data=(X_test,y_test),batch_size=20,callbacks=callback_list)


Epoch 1/30
798/800 [============================>.] - ETA: 0s - loss: 0.6856 - accuracy: 0.5568 F1 Score : 0.6434216643421664 ,  AUC Score : 0.6168522412578854 

Epoch 00001: val_accuracy improved from -inf to 0.61650, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-01-0.6165.hdf5
 learning rate :  0.001
800/800 [==============================] - 2s 2ms/step - loss: 0.6856 - accuracy: 0.5569 - val_loss: 0.6724 - val_accuracy: 0.6165
Epoch 2/30
771/800 [===========================>..] - ETA: 0s - loss: 0.6440 - accuracy: 0.6379 F1 Score : 0.6613939693524469 ,  AUC Score : 0.6575650656925802 

Epoch 00002: val_accuracy improved from 0.61650 to 0.65750, saving model to /content/drive/My Drive/Colab Notebooks/question 20/model_save/weights-02-0.6575.hdf5
 learning rate :  0.001
800/800 [==============================] - 2s 2ms/step - loss: 0.6432 - accuracy: 0.6381 - val_loss: 0.6166 - val_accuracy: 0.6575
Epoch 3/30
798/800 [=========================

In [ ]:
!kill 3029

/bin/bash: line 0: kill: (3029) - No such process


In [ ]:
%tensorboard --logdir logs/fit/
#to launch tensorboard
#screenshots have been attached in the seperate pdf file